In [181]:
import pandas as pd
from pathlib import Path
import os
from statistics import mode
import numpy as np

## Decompose: final label -> individual

In [182]:
all_decompose = {}
label = 0

# All combined
for m in range(3):
    for g in range(2):
        for a in range(3):
            all_decompose[label] = ':'.join([str(m),str(g),str(a)])
            label += 1
            
# mask & gender

mask_gender_decompose = {}
label = 0

for m in range(3):
    for g in range(2):
            mask_gender_decompose[label] = ':'.join([str(m),str(g)])
            label += 1

# Age 
age_decompose = {
    0: 0,
    1: 0,
    2: 1,
    3: 1,
    4: 1,
    5: 2
}

## individual -> final label

In [183]:
# All combined (mask, gender, age)
label = 0
label_trsfm = {}

for m in range(3):
    for g in range(2):
        for a in range(3):
            label_trsfm[':'.join([str(m),str(g),str(a)])] = label
            label += 1

## csv directory

In [184]:
all_csv_dir = Path('./out/all_hardvoting')
mask_gender_csv_dir = Path('./out/mask_gender_hardvoting')
age_csv_dir = Path('./out/age_hardvoting')

In [185]:
all_csv_names = os.listdir(all_csv_dir)
mask_gender_csv_names = os.listdir(mask_gender_csv_dir)
age_csv_names = os.listdir(age_csv_dir)

In [186]:
all_df_list = []
mask_gender_df_list = []
age_df_list = []

In [187]:
DH_age_csv = pd.read_csv('./out/DH_age.csv')

In [188]:
for csv in all_csv_names:
    if not csv.endswith('.csv'):
        continue
    all_df_list.append(pd.read_csv(all_csv_dir / csv))
    
for csv in mask_gender_csv_names:
    if not csv.endswith('.csv'):
        continue
    mask_gender_df_list.append(pd.read_csv(mask_gender_csv_dir / csv))
    
for csv in age_csv_names:
    if not csv.endswith('.csv'):
        continue
    age_df_list.append(pd.read_csv(age_csv_dir / csv))

In [189]:
final_label = []
for idx in range(12600):
    freq = {
        'mask': [],
        'gender': [],
        'age': []
    }
    
    # For All (mask, gender, age)
    for df_idx in range(len(all_df_list)):
        combined_label = all_df_list[df_idx].loc[idx, 'ans']
        mask, gender, age = all_decompose[combined_label].split(':')
        freq['mask'].append(mask)
        freq['gender'].append(gender)
        freq['age'].append(age)
    
    # For Mask & Gender
    for df_idx in range(len(mask_gender_df_list)):
        combined_label = mask_gender_df_list[df_idx].loc[idx, 'ans']
        mask, gender = mask_gender_decompose[combined_label].split(':')
        freq['mask'].append(mask)
        freq['gender'].append(gender)
        
    # For Age
    for df_idx in range(len(age_df_list)):
        age6 = age_df_list[df_idx].loc[idx, 'ans']
        age3 = age_decompose[age6]
        freq['age'].append(age3)
        
    # DH age
    freq['age'].append(DH_age_csv.loc[idx, 'ans'])
    
    mask_mode = mode(freq['mask'])
    gender_mode = mode(freq['gender'])
    age_mode = mode(freq['age'])
    final_label.append(label_trsfm[":".join([mask_mode, gender_mode, age_mode])])
    
    # Print final lable for current image
    # print("Final label: {}".format(label_trsfm[":".join([mask_mode, gender_mode, age_mode])]))

In [193]:
submitted_csv = pd.read_csv('submitted.csv')

In [198]:
sum(np.array(list(submitted_csv['ans'])) != np.array(final_label))

0